In [9]:
import spacy
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN, KMeans

import solrhandler as sh
import clusterer as cls
import topicdeterminator as td
from service_clustering import ServiceClustering

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
#query = "anmeldung"
#target_service = 120686

In [53]:
# Import eval.csv
import os
file = os.path.realpath("../../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
#eval = eval.head(500)
eval.head(20)

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...
5,328200,20220131--quantEvalAllExcel.xlsx,24,Wo kann ich mich nach Corona Infektion freites...,6,True,Infektionskrankheiten melden - für Gemeinschaf...
6,330073,20220131--quantEvalAllExcel.xlsx,28,Ab wann ist man bei Johnson & Johnson geboostert,5,True,Impfung gegen Corona (SARS-CoV-2)
7,120702,20220131--quantEvalAllExcel.xlsx,36,Genesenen Bescheinigung,6,True,Meldebescheinigung beantragen
8,330150,20220131--quantEvalAllExcel.xlsx,39,"welche medikamente kann man nehmen, wenn man c...",6,True,Arzneimittel - zur Sachkundeprüfung für den Ha...
9,330174,20220131--quantEvalAllExcel.xlsx,47,ab 15.1. ist für Gäste von Restaurants Nachwe...,6,True,Gaststättengewerbe - zum Unterrichtungsnachwei...


In [4]:
#eval.iloc[2]["initialQuestion"]

Funktionen

In [48]:
def findCorrectAnswer(chatbot,targetService):
    
    # Find Target Cluster
    clusteredColumn = "cluster_label"
    service = chatbot.df_current.loc[chatbot.df_current["id"]==str(targetService)]
    if 'cluster_label' not in service.columns:
        return None
    
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]
    
    if targetCluster == chatbot.max_cluster_label_current:
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df_current.reset_index()
    try:
        rank = temp.index[temp["id"]==str(targetService)].values[0]
    except IndexError as e:
        return float("NaN")
    return rank + 1

Logging

In [60]:
class Logger:
    
    def __init__(self, eval_df):
        self.liste = []
        self.eval_df = eval_df
        self.chatbot = None
    
    def getDataFrame(self): # TODO
        pass
    
    def setChatbot(self, chatbot):
        self.chatbot = chatbot
    
    def log(self, ID, t, row, answer = None, initialQuery = False): 
        target_service = row["documentId"]
        #print(ID)
        #print(t)
        #print(answer)
       
        dialogId = row["dialogId"]
        file = row["file"]
        
        service_name = row["name"]
        query = row["initialQuestion"]
        if initialQuery==True:
            nResults = len(self.chatbot.df_current.index)
            rank = getRank(self.chatbot,target_service)# getRank(target_service)
            
            row = (file, dialogId, ID, t, service_name, query, "initialQuery", None, rank, nResults)
        else:
            if answer is not None:

                
                nResults = len(self.chatbot.df_current.index)
                question = self.chatbot.generateQuestion()
                answer = answer

                rank = getRank(self.chatbot,target_service)# getRank(target_service)

                row = (file, dialogId, ID, t,  service_name, query, question, answer, rank, nResults)

            else:
                row = (file, dialogId, ID, t, service_name, query, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

TEST

Execution

In [10]:
kmeans = KMeans(n_clusters=2)
chatbot= ServiceClustering(query, kmeans, max_results=3)

In [6]:
# Initialisierung Komponenten Chatbot
#solrhandler = sh.SolrHandler(1000)
#clusterer = cls.Clusterer()
#tpc_dterminator = td.TopicDeterminator()

from sklearn.cluster import DBSCAN


In [ ]:
# Experiment durchlaufen und Loggen
lg = Logger(eval_df = eval)   

for (ID, row) in eval.iterrows():
    print(ID)
    query = row["initialQuestion"]
    target_service = row["documentId"]
    t = 0
    
    # Chatbot initialisieren
    try: # 1.3
        chatbot =  ServiceClustering(query, kmeans, max_results=1)
    
    # Fehler überspringen: eg. kein solr output
    except Exception as e: 
        if(str(e) == 'response'): # TODO ??
            print(e)
            next
        elif(str(e) == 'no solr output'):
            print(e)
            next
        elif(str(e) == 'cluster_label'):
            print(e)
            next    
        else:
            print(e)
            next
            
            
    lg.setChatbot(chatbot)

    
    answer = findCorrectAnswer(chatbot, target_service)
    
    if answer is None: # service nicht in resultset, überspringen
        next
    lg.log(ID,t, row, answer, initialQuery = True)
    
    while not chatbot.isFinished():#chatbot.refineResultset(answer, recluster = True):

        # log
        t += 1
        
#         print(ID)
#         print(t)
#         print(np.unique(chatbot.df["ssdsLemma_cluster"]))
#         print("")
        
        chatbot.refineResultset(answer)
        answer = findCorrectAnswer(chatbot, target_service)
        
        if answer is None: 
            break
        lg.log(ID, t, row,answer)
        
        #print(chatbot.clusterer.clustering_algorithm.eps)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
'response'
15
16
17
Wrong number of items passed 0, placement implies 1
18
19
20
21
22
23
24
25
26
27
28
29
30
Wrong number of items passed 0, placement implies 1
31
32
33
34
35
36
37
38
39
40
41
42
43
44
Wrong number of items passed 0, placement implies 1
45
46
47
48
49
50
'response'
51
52
53
54
55
56
57
58
59
60
61
62
Wrong number of items passed 0, placement implies 1
63
64
65
66
67
Wrong number of items passed 0, placement implies 1
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
Wrong number of items passed 0, placement implies 1
86
87
Wrong number of items passed 0, placement implies 1
88
Wrong number of items passed 0, placement implies 1
89
90
91
92
93
94
95
96
97
98
99
100
Wrong number of items passed 0, placement implies 1
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
Wrong number of items passed 0, placement implies 1
117
118
'response'
119
120
121
122
123
124
125
126
127
128
129
130
Wrong number of items passed 0, p

In [ ]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId", "ID","t", "name", "query", "question", "answer", "rank", "nResult"])
log_df.to_csv("service_clus_kmeans_n1.csv")

In [47]:
eval.iloc[5,:]

documentId                                                    328200
file                                20220131--quantEvalAllExcel.xlsx
dialogId                                                          24
initialQuestion    Wo kann ich mich nach Corona Infektion freites...
suggestionCount                                                    6
directlyFound                                                   True
name               Infektionskrankheiten melden - für Gemeinschaf...
Name: 5, dtype: object

In [52]:
log_df

,file,dialogId,ID,t,name,query,question,answer,rank,nResult
0,20220131--quantEvalAllExcel.xlsx,4,0,0,Änderung/Wechsel der Hauptwohnung,Ummeldung,initialQuery,None,3.0,7
1,20220131--quantEvalAllExcel.xlsx,4,0,1,Änderung/Wechsel der Hauptwohnung,Ummeldung,Ist Ihre Frage zum Thema Ummeldung ?,True,2.0,6
2,20220131--quantEvalAllExcel.xlsx,4,0,2,Änderung/Wechsel der Hauptwohnung,Ummeldung,Ist Ihre Frage zum Thema Ummeldung ?,True,2.0,3
3,20220131--quantEvalAllExcel.xlsx,4,1,0,Anmeldung einer Wohnung,Ummelden,initialQuery,None,NaN,7
4,20220131--quantEvalAllExcel.xlsx,19,2,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,initialQuery,None,4.0,13
...,...,...,...,...,...,...,...,...,...,...
123,20220131--quantEvalAllExcel.xlsx,186,33,3,Kraftfahrzeug ummelden – nach einem Umzug nach...,Kfz anmeldung,Ist Ihre Frage zum Thema Fahrzeugzulassung ?,True,22.0,28
124,20220131--quantEvalAllExcel.xlsx,186,33,4,Kraftfahrzeug ummelden – nach einem Umzug nach...,Kfz anmeldung,Ist Ihre Frage zum Thema Fahrzeugzulassung ?,False,12.0,16
125,20220131--quantEvalAllExcel.xlsx,186,33,5,Kraftfahrzeug ummelden – nach einem Umzug nach...,Kfz anmeldung,Ist Ihre Frage zum Thema Ummeldung ?,True,4.0,5
126,20220131--quantEvalAllExcel.xlsx,186,33,6,Kraftfahrzeug ummelden – nach einem Umzug nach...,Kfz anmeldung,Ist Ihre Frage zum Thema Kennzeichen ?,False,3.0,4


Log Analyzer

In [10]:
import Modules.log_analyzer as lo_an

In [11]:
log_analyzer=lo_an.Log_Analyzer(log_df)
delta,services,questions,means=log_analyzer.run_it()

ValueError: too many values to unpack (expected 4)

In [ ]:
log_df.head(40)

In [ ]:
means

Statistics ( Alt)

In [ ]:
# avg Rank Improvement in turn t
log_df["delta_Rank"] = log_df.groupby(["ID"])["rank"].diff()
#log_df.groupby(["t"])["rank_improvement"].mean()

In [ ]:
# avg nSuggestions Improvement in turn t
log_df["delta_nResults"] = log_df.groupby(["ID"])["nResults"].diff()
#log_df.groupby(["t"])["sugg_improvement"].mean()

In [ ]:
# MRR ohne cluster refinement
#df = log_df.loc[log_df["t"] == 1]
#MRR_ohne = 1/(df["rank"].mean())
#MRR_ohne

In [ ]:
# MRR mit cluster refinement
#df = log_df.groupby(["ID"]).last("t")
#MRR_mit = 1/(df["rank"].mean())
#MRR_mit
#df.head()

In [ ]:
# avg Turns
print(log_df["t"].mean())

# bei durchschnittlich nResults
print(log_df.groupby(["ID"]).last("t")["nResults"].mean())

In [ ]:
log_df.head(20)